# Java 2 — Tokenizimi & Embeddings

**Objektivat:**
- Të kuptohet chunking dhe ndikimi i tij.
- Të gjenerohen embeddings për chunks.
- Të testohet ngjashmëria semantike për pyetje tipike.

**Detyra (Assignment):**
- Shndërro 3 PDF në tekst.
- Krijo chunks (heading-based + size-based) dhe krahaso.
- Gjenero embeddings dhe kthe top-5 chunks për 5 pyetje.

---

# Doctrine & SOP Assistant (Fiktiv) — Google Colab Notebook

**Kujdes / Safety (e detyrueshme):**
- Ky projekt është *arsimor* dhe përdor dokumente **fiktive, të paklasifikuara**.
- Asistenti **nuk** jep këshilla taktike/operacionale.
- Nëse informacioni nuk gjendet në dokumente → **refuzim i sigurt** (“Nuk mbulohet nga dokumentet e disponueshme.”).

---

## 📁 Struktura e të dhënave (rekomandim)

Vendosni PDF-të në:
- `data/raw_pdfs/`

Emrat e rekomanduar (shembull):
- `AAF-TRN-001_SOP_e_Stërvitjes.pdf`
- `AAF-SAFE-010_Rregullat_e_Sigurisë_në_Poligon.pdf`
- `AAF-LOG-004_Siguria_dhe_Kontrolli_i_Mjeteve.pdf`
- `AAF-CMD-007_Zinxhiri_Komandues_dhe_Raportimi_i_Incidenteve.pdf`
- `AAF-GRD-002_Procedurat_e_Sherbimit_te_Rojes.pdf`
- `NATO_Permbledhje_Doktrinore_Edukative.pdf`
- `AAF-IDX-000_Indeksi_Doktrinor_dhe_Procedural.pdf`
- `AAF-ADM-012_Rregulla_Administrative_dhe_Arkivimi.pdf`
- `AAF-RPT-009_Udhezime_te_Pergjithshme_per_Raportim.pdf`


## 0) Setup & Import

In [ ]:
!pip -q install sentence-transformers chromadb pypdf faiss-cpu rapidfuzz

import os, re, json, math
from pathlib import Path


## 1) Ngarko PDF-të (zgjidh një mënyrë)

### Opsioni A — Upload direkt në Colab

In [ ]:
from google.colab import files
uploaded = files.upload()  # zgjidh PDF-të nga kompjuteri
os.makedirs("data/raw_pdfs", exist_ok=True)
for fn in uploaded.keys():
    Path(fn).replace(Path("data/raw_pdfs")/fn)
print("U ngarkuan:", list(uploaded.keys()))


### Opsioni B — Mount Google Drive (rekomanduar për projekte)

In [ ]:
# nga google.colab import drive
# drive.mount('/content/drive')
# Pastaj kopjoni/ndryshoni rrugën sipas vendit ku i keni ruajtur PDF-të, p.sh.:
# !mkdir -p data/raw_pdfs
# !cp -n /content/drive/MyDrive/doctrine_sop_dataset/raw_pdfs/*.pdf data/raw_pdfs/


## 2) Konvertimi PDF → tekst

In [ ]:
from pypdf import PdfReader

def pdf_to_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    parts = []
    for i, page in enumerate(reader.pages):
        txt = page.extract_text() or ""
        parts.append(txt)
    return "\n".join(parts)

pdf_dir = Path("data/raw_pdfs")
pdfs = sorted(pdf_dir.glob("*.pdf"))
print("PDF count:", len(pdfs))
for p in pdfs[:5]:
    print("-", p.name)


## 3) Chunking

In [ ]:
import re

def simple_section_chunk(text: str, max_chars: int = 2500, overlap: int = 300):
    # chunk by headings (very simple heuristic) then by size
    text = re.sub(r'\r\n', '\n', text)
    lines = [ln.strip() for ln in text.split('\n')]
    # treat lines that look like headings: start with digit + '.' or all caps short
    blocks = []
    cur = []
    for ln in lines:
        if re.match(r'^\d+\.', ln) and cur:
            blocks.append("\n".join(cur).strip())
            cur = [ln]
        else:
            if ln:
                cur.append(ln)
    if cur:
        blocks.append("\n".join(cur).strip())

    chunks = []
    for b in blocks:
        if len(b) <= max_chars:
            chunks.append(b)
        else:
            start = 0
            while start < len(b):
                end = min(len(b), start + max_chars)
                chunks.append(b[start:end])
                start = max(0, end - overlap)
    return [c for c in chunks if c.strip()]


In [ ]:
texts = {}
for p in sorted(Path("data/raw_pdfs").glob("*.pdf")):
    texts[p.stem] = pdf_to_text(str(p))

sample_key = next(iter(texts.keys())) if texts else None
if sample_key:
    chunks = simple_section_chunk(texts[sample_key])
    print("Doc:", sample_key, "chunks:", len(chunks))
    print(chunks[0][:500])
else:
    print("No PDFs found. Upload/mount first.")


## 4) Embeddings & Similarity Search (manual)

In [ ]:
from sentence_transformers import SentenceTransformer

EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL)
print("Loaded:", EMB_MODEL)


In [ ]:
import numpy as np

def top_k(query, chunks, k=5):
    qv = embedder.encode([query], normalize_embeddings=True)[0]
    cvs = embedder.encode(chunks, normalize_embeddings=True)
    sims = np.dot(cvs, qv)
    idx = np.argsort(-sims)[:k]
    return [(int(i), float(sims[i]), chunks[i][:300]) for i in idx]

if sample_key:
    q = "Cila komandë jepet në rast emergjence në poligon?"
    res = top_k(q, chunks, k=5)
    for i, s, snippet in res:
        print(i, round(s,3), snippet.replace("\n"," ")[:260])


## 5) Dorëzim

- Jep 5 pyetje dhe top-3 chunks për secilën.
- Shpjego shkurt cilin chunking zgjedh dhe pse.